In [ ]:
# importing libraries
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import matplotlib.pyplot as plt

Datadirectory = "DataSet/"
Classes = ["NoFaceMask", "WithFaceMask"]

training_Data = []

# definition of function
def create_training_Data():
    for category in Classes:
        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (img_size, img_size))
                training_Data.append([new_array, class_num])
            except Exception as e:
                pass
            
# calling function
create_training_Data()

# using pre-trained model
model = tf.keras.applications.mobilenet.MobileNet()

# reviewing mpre-trained model summary
model.summary()

# Transfer Learning - tuning weights will start from last checkpoints
base_input = model.layers[0].input
base_output = model.layers[-4].output

Flat_layer = layers.Flatten()(base_output)
final_output = layers.Dense(1)(Flat_layer)
finalOutput = layers.Activation('sigmoid')(final_output)

new_model = keras.Model(inputs = base_input, outputs = final_output)

# reviewing new model summary
new_model.summary()

# SETTINGS FOR BINARY CLASSIFICATION (FACE MASK / WITHOUT FACE MASK)
new_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

new_model.fit(X, Y, epochs = 1, validation_split = 0.1)

# saving the model
new_model.save("model1.h5")